In [1]:
import pandas as pd
import csv
import json
import requests
from datetime import datetime
import itertools
from collections import Counter

def getDataFromPagerApi(url, per_page = None):
    data = []
    end = False
    page = 1
    while not end:
        response = requests.get(url + '?page=' + str(page) + ('&per_page='+str(per_page) if per_page else '')).json()
        data += response['data']
        if page >= response['pages']:
            break
        page += 1
    return data

In [209]:
date_ = ''
API_URL = 'https://data.parlameter.si/v1'
url = API_URL + '/getVotesTableExtended'
data = getDataFromPagerApi(url)
data = pd.DataFrame(data)
url = API_URL + '/getMPs/' + date_
mps = requests.get(url).json()
members = [mp['id'] for mp in mps]
url = API_URL + '/getMembersOfPGsOnDate/' + date_
memsOfPGs = requests.get(url).json()
url = API_URL + '/getAllPGs/' + date_
pgs = requests.get(url).json()

In [210]:
coalition = requests.get(API_URL + '/getCoalitionPGs').json()['coalition']
orgs = requests.get(API_URL + '/getAllPGsExt/')
data['option_ni'] = 0
data['option_za'] = 0
data['option_proti'] = 0
data['option_kvorum'] = 0
data.loc[data['option'] == 'ni', 'option_ni'] = 1
data.loc[data['option'] == 'za', 'option_za'] = 1
data.loc[data['option'] == 'proti', 'option_proti'] = 1
data.loc[data['option'] == 'kvorum', 'option_kvorum'] = 1
data['voter_unit'] = 1
data['is_coalition'] = 0
data.loc[data['voterparty'].isin(coalition), 'is_coalition'] = 1

#za proti ni kvorum
all_votes = data.groupby('vote_id').sum()
m_to_p = {i['id']: i['party_id'] for i in mps}
mppgs = pd.DataFrame(m_to_p.items(), columns=['voter', 'voterparty'])

In [211]:
#Get ballots of last members party
keys = ['voter', 'voterparty']
i1 = mppgs.set_index(keys).index
i2 = data.set_index(keys).index
data2 = data[i2.isin(i1)]

In [125]:
data.head()

,acronym,date,id,option,orgvoter,result,session_id,tags,text,vote_id,voter,voterparty,option_ni,option_za,option_proti,option_kvorum,voter_unit,is_coalition
0,NSI,2016-06-14 20:00:56,653592,za,NaN,False,5576,Odbor za finance in monetarno politiko,Zakon o reševanju in prisilnem prenehanju bank...,7265,32,6,0,1,0,0,1,0
1,SDS,2016-06-14 20:00:56,653566,kvorum,NaN,False,5576,Odbor za finance in monetarno politiko,Zakon o reševanju in prisilnem prenehanju bank...,7265,2,5,0,0,0,1,1,0
2,SDS,2016-06-14 20:00:56,653573,ni,NaN,False,5576,Odbor za finance in monetarno politiko,Zakon o reševanju in prisilnem prenehanju bank...,7265,10,5,1,0,0,0,1,0
3,SDS,2016-06-14 20:00:56,653575,kvorum,NaN,False,5576,Odbor za finance in monetarno politiko,Zakon o reševanju in prisilnem prenehanju bank...,7265,12,5,0,0,0,1,1,0
4,SDS,2016-06-14 20:00:56,653584,ni,NaN,False,5576,Odbor za finance in monetarno politiko,Zakon o reševanju in prisilnem prenehanju bank...,7265,23,5,1,0,0,0,1,0


In [249]:
mps[0]['acronym']
def getPartyBallot(row):
    """
    using for set ballot of party:

    methodology: ignore not_present
    """
    stats = {'za': row['option_za'],
             'proti': row['option_proti'],
             'kvorum': row['option_kvorum']}
             #'ni': row['option_ni']}
    if max(stats.values()) == 0:
        return None
    max_ids = [key for key, val in stats.iteritems() if val == max(stats.values())]
    #if len(max_ids) > 1:
    #    return None
    #return max_ids[0]
    return ','.join(max_ids)

partyBallots = data.groupby(['vote_id',
                             'voterparty']).sum().apply(lambda row: getPartyBallot(row), axis=1)
partys = partyBallots.reset_index()
partys = partys.rename(columns = {0:'partyoption'})

In [222]:
partyBallots.head()

vote_id  voterparty
4548     1              proti
         2             kvorum
         3              proti
         5                 za
         6                 za
dtype: object

In [250]:
result = pd.merge(data2, partys, on=['vote_id', 'voterparty'])
#remove ni option
result = result[result.option != 'ni']

In [251]:
members_vote_count = result[['voter','voter_unit']].groupby('voter').count()

In [236]:
def is_equal(row):
    if row['partyoption'] and row['option'] in row['partyoption']:
        return 1
    else:
        return 2

In [252]:
#equal_indexes = result[['option', 'partyoption']].apply(pd.Series.nunique, axis=1).reindex()
equal_indexes = result[['option', 'partyoption']].apply(lambda x: is_equal(x), axis=1).reindex()
result['ivan'] = result[['option', 'partyoption']].apply(lambda x: is_equal(x), axis=1).reindex()

In [253]:
result[result['voter']==4]


,acronym,date,id,option,orgvoter,result,session_id,tags,text,vote_id,voter,voterparty,option_ni,option_za,option_proti,option_kvorum,voter_unit,is_coalition,partyoption,ivan
35,IMNS,2016-06-14 20:00:56,653568,za,NaN,False,5576,Odbor za finance in monetarno politiko,Zakon o reševanju in prisilnem prenehanju bank...,7265,4,2,0,1,0,0,1,0,za,1
170,IMNS,2016-06-14 20:01:32,653478,kvorum,NaN,True,5576,Odbor za finance in monetarno politiko,Zakon o reševanju in prisilnem prenehanju bank...,7264,4,2,0,0,0,1,1,0,kvorum,1
239,IMNS,2016-06-21 21:50:30,444732,za,NaN,True,5576,"Odbor za infrastrukturo, okolje in prostor",Uradno prečiščeno besedilo (Pomorskega zakonika),4943,4,2,0,1,0,0,1,0,za,1
418,IMNS,2016-06-21 22:10:39,444192,kvorum,NaN,True,5576,Odbor za finance in monetarno politiko,Zakon o trošarinah - Glasovanje o zakonu v celoti,4937,4,2,0,0,0,1,1,0,kvorum,1
504,IMNS,2016-06-21 22:09:16,444282,za,NaN,True,5576,Mandatno-volilna komisija,Sklep o razrešitvi podpredsednika in imenovanj...,4938,4,2,0,1,0,0,1,0,za,1
578,IMNS,2016-06-21 22:08:23,444372,kvorum,NaN,True,5576,Mandatno-volilna komisija,Sklep o imenovanju članov Sveta za elektronske...,4939,4,2,0,0,0,1,1,0,"kvorum,za",1
680,IMNS,2016-06-21 22:07:39,444462,kvorum,NaN,True,5576,Mandatno-volilna komisija,Predlog za izvolitev v sodniško funkcijo na so...,4940,4,2,0,0,0,1,1,0,kvorum,1
762,IMNS,2016-06-21 22:07:06,444552,kvorum,NaN,True,5576,Mandatno-volilna komisija,Predlog za izvolitev v sodniško funkcijo na so...,4941,4,2,0,0,0,1,1,0,kvorum,1
837,IMNS,2016-06-21 22:06:34,444642,kvorum,NaN,True,5576,Mandatno-volilna komisija,Predlog za izvolitev v sodniško funkcijo na so...,4942,4,2,0,0,0,1,1,0,kvorum,1
929,IMNS,2016-06-21 21:49:43,444822,za,NaN,True,5576,Proceduralna glasovanja,Proceduralni predlog za prestavitev točke (Ura...,4944,4,2,0,1,0,0,1,0,za,1


In [254]:
out = pd.concat([result, equal_indexes], axis=1).rename(columns = {0: 'equal_vote'})
out = out[out['equal_vote']==1]

In [255]:
members_equal_count = out[['voter', 'equal_vote']].groupby('voter').sum()

In [205]:
members_equal_count.head()

,equal_vote
voter,
2,1465
3,2167
4,1136
5,853
7,652


In [188]:
members_vote_count.head()

,voter_unit
voter,
2,1478
3,2172
4,1471
5,892
7,704


In [256]:
final = pd.concat([members_equal_count, members_vote_count], axis=1)

In [257]:
final['percent'] = final.apply(lambda x: float(x['equal_vote'])/x['voter_unit']*100.0, axis=1)

In [260]:
final.index.values

array([   2,    3,    4,    5,    7,    9,   10,   11,   12,   14,   15,
         16,   17,   18,   19,   21,   22,   23,   24,   25,   26,   27,
         29,   30,   31,   32,   33,   34,   35,   36,   37,   39,   40,
         41,   42,   43,   44,   45,   47,   48,   49,   50,   51,   52,
         53,   54,   55,   57,   58,   59,   60,   61,   62,   63,   64,
         65,   66,   67,   68,   69,   70,   71,   72,   73,   74,   75,
         76,   77,   78,   79,   80,   81,   82,   83,   84,   85,   86,
         87,   88,   89,   91,   92,   95,   96, 1354, 1355, 1356, 1357,
       2933, 2934])

In [277]:
for index, row in final.iterrows():
    print index, row['percent']

2 99.1880920162
3 99.7697974217
4 100.0
5 95.8520179372
7 98.2954545455
9 98.6688851913
10 98.6708365911
11 99.4272076372
12 99.3980436418
14 99.8406797663
15 100.0
16 99.2776057792
17 99.2418498863
18 89.4660894661
19 98.8958990536
21 99.399098648
22 96.4247598719
23 98.8878591288
24 100.0
25 96.4128382631
26 99.5977473854
27 99.3537964459
29 95.6384655807
30 98.753117207
31 100.0
32 99.1379310345
33 99.0745250852
34 98.2862903226
35 99.1607488702
36 93.9672801636
37 98.3033419023
39 99.730458221
40 99.7185741088
41 97.1721111653
42 97.9153506001
43 99.6187363834
44 99.5504495504
45 97.0404984424
47 96.4019851117
48 99.7604216579
49 98.9010989011
50 95.4372623574
51 99.6
52 96.9618949537
53 97.4050632911
54 99.4074074074
55 98.8150098749
57 99.4126284875
58 95.7250144425
59 97.8217821782
60 99.6031746032
61 96.5829145729
62 95.6278026906
63 99.8706338939
64 96.7663209274
65 99.0965556183
66 98.5384615385
67 99.755859375
68 97.9802724284
69 98.4034833091
70 99.9048072347
71 99.49003245